In [20]:
import torch
%run loader.ipynb

In [1]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

NameError: name 'torch' is not defined

In [22]:
def train_vae(train_loader, net, optimizer, device=device):
    """
    Trains variational autoencoder network for one epoch in batches.
    Args:
        train_loader: Data loader for training set.
        net: Neural network model.
        optimizer: Optimizer (e.g. SGD).
        device: whether the network runs on cpu or gpu
    """
    avg_loss = 0

    # iterate through batches
    for i, data in enumerate(train_loader):
        if i % 10 == 0:
            print(i)
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        #rint(inputs.shape)
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs, mean, logvar = net(inputs, labels)
        # loss = ((inputs - outputs) ** 2).sum() + net.encoder.kl + (inputs - sr).abs().sum()
        l = loss(inputs, outputs, mean, logvar).to(device)
        l.backward()
        optimizer.step()

        # keep track of loss and accuracy
        avg_loss += l / len(labels)
    return avg_loss / len(train_loader)
def loss(X, X_hat, mean, logvar):
    reconstruction_loss = BCE_loss(X_hat, X)
    KL_divergence = 0.5 * torch.sum(-1 - logvar + torch.exp(logvar) + mean ** 2)
    return reconstruction_loss + KL_divergence